# Imports

In [1]:
import pandas as pd
import os
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
from scipy.stats import mode
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression

# Loading the Dataframe

In [2]:
pwd

'/home/jonathan/code/rafabertolace/OnThePitch/notebooks'

## Merging the Seasons csv files (2019-2020 untill 2021-2022)

In [3]:
print(os.listdir('../raw_data/All'))

['I1 (3).csv', 'N1 (1).csv', 'F1.csv', 'B1 (2).csv:Zone.Identifier', 'N1 (2).csv', 'P1 (6).csv', 'F1 (2).csv', 'P1 (4).csv', 'SC0 (2).csv', 'G1 (2).csv', 'D1.csv', 'SP1 (1).csv', 'E0 (2).csv', 'I1 (1).csv:Zone.Identifier', 'SC0 (1).csv', 'F1.csv:Zone.Identifier', 'D1 (2).csv:Zone.Identifier', 'I1 (3).csv:Zone.Identifier', 'SP1.csv', 'D1.csv:Zone.Identifier', 'SC0.csv', 'I1 (2).csv', 'Turkey_2021_2022.csv.csv', 'B1 (2).csv', 'B1 (1).csv', 'F1 (1).csv', 'Turkey_2019_2020.csv.csv', 'SP1 (2).csv:Zone.Identifier', 'Turkey_2020_2021.csv.csv', 'SP1.csv:Zone.Identifier', 'SP1 (2).csv', 'G1.csv', 'SC0 (1).csv:Zone.Identifier', 'E0.csv', 'SC0.csv:Zone.Identifier', 'I1 (1).csv', 'F1 (2).csv:Zone.Identifier', 'N1.csv', 'SC0 (2).csv:Zone.Identifier', 'D1 (2).csv', 'D1 (1).csv', 'I1 (2).csv:Zone.Identifier', 'B1 (1).csv:Zone.Identifier', 'E0 (1).csv', 'F1 (1).csv:Zone.Identifier', 'P1 (5).csv', 'G1 (1).csv', 'D1 (1).csv:Zone.Identifier', 'B1.csv:Zone.Identifier', 'B1.csv', 'SP1 (1).csv:Zone.Identifi

In [4]:
files = [file for file in os.listdir('../raw_data/All') if file.endswith('.csv')]
data = pd.DataFrame()

for file in files:
    df = pd.read_csv('../raw_data/All/' + file)
    df['country']=str(file)[0:2]
    data = pd.concat([data, df])

In [5]:
for col_name in data.columns: 
    print(col_name)

Div
Date
Time
HomeTeam
AwayTeam
FTHG
FTAG
FTR
HTHG
HTAG
HTR
HS
AS
HST
AST
HF
AF
HC
AC
HY
AY
HR
AR
B365H
B365D
B365A
BWH
BWD
BWA
IWH
IWD
IWA
PSH
PSD
PSA
WHH
WHD
WHA
VCH
VCD
VCA
MaxH
MaxD
MaxA
AvgH
AvgD
AvgA
B365>2.5
B365<2.5
P>2.5
P<2.5
Max>2.5
Max<2.5
Avg>2.5
Avg<2.5
AHh
B365AHH
B365AHA
PAHH
PAHA
MaxAHH
MaxAHA
AvgAHH
AvgAHA
B365CH
B365CD
B365CA
BWCH
BWCD
BWCA
IWCH
IWCD
IWCA
PSCH
PSCD
PSCA
WHCH
WHCD
WHCA
VCCH
VCCD
VCCA
MaxCH
MaxCD
MaxCA
AvgCH
AvgCD
AvgCA
B365C>2.5
B365C<2.5
PC>2.5
PC<2.5
MaxC>2.5
MaxC<2.5
AvgC>2.5
AvgC<2.5
AHCh
B365CAHH
B365CAHA
PCAHH
PCAHA
MaxCAHH
MaxCAHA
AvgCAHH
AvgCAHA
country
Referee
Unnamed: 105


# Features Engineering

### Number of Goals, Over and Under

In [6]:
# total number of goals = goals from the home team + goals from visiting team
data['nb_goals']=data['FTHG']+data['FTAG']

# boolean: true or false regarding whether they were more than 2.5 goals
data['over_2.5_goals']=data['nb_goals']>2.5

# boolean: true or false regarding whether they were less than 2.5 goals
data['under_2.5_goals']=data['nb_goals']<2.5

### Payout

  Payout of betting on over/under 2.5 goals: we get 0 if we lose the bet, we get the Avg if we win the bet (Avg = market average of the odds)


#### Payout Opening

In [7]:
# payout under 2.5 for Average OPENING odds
data['payout_avg_under_2.5'] = data['under_2.5_goals']*data['Avg<2.5']

# payout over 2.5 for Average OPENING odds
data['payout_avg_over_2.5'] = data['over_2.5_goals']*data['Avg>2.5']

#payout UNDER 2.5 for PINACLE specifically
data['payout_under_2.5_pinacle'] = data['under_2.5_goals']*data['P<2.5']

#payout OVER 2.5 for PINACLE specifically
data['payout_over_2.5_pinacle'] = data['over_2.5_goals']*data['P>2.5']

#payout UNDER 2.5 for 365 specifically
data['payout_under_2.5_365'] = data['under_2.5_goals']*data['B365<2.5']

#payout OVER 2.5 for 365 specifically
data['payout_over_2.5_365'] = data['over_2.5_goals']*data['B365>2.5']

#### Payout Closing

In [8]:
# payout under 2.5 for Average CLOSING odds
data['payout_avg_under_closing_2.5'] = data['under_2.5_goals']*data['AvgC<2.5']

# payout over 2.5 for Average CLOSING odds
data['payout_avg_over_closing_2.5'] = data['over_2.5_goals']*data['AvgC>2.5']

#payout UNDER 2.5 for PINACLE closing ddds specifically
data['payout_under_2.5_pinacle_closing'] = data['under_2.5_goals']*data['PC<2.5']

#payout OVER 2.5 for PINACLE closing odds specifically
data['payout_over_2.5_pinacle_closing'] = data['over_2.5_goals']*data['PC>2.5']

#payout UNDER 2.5 for 365 closing odds specifically
data['payout_under_2.5_365_closing'] = data['under_2.5_goals']*data['B365C<2.5']

#payout OVER 2.5 for 365 closing odds specifically
data['payout_over_2.5_365_closing'] = data['over_2.5_goals']*data['B365C>2.5']

### Implied Probability

#### Implied Probability Opening

In [9]:
#Implied Probability UNDER 2.5 goals for for overall market opening odds (Avg) 
data['Implied Probability <2.5 avg']=1/data['Avg<2.5']*100

#Implied Probability OVER 2.5 goals for for overall market opening odds (Avg) 
data['Implied Probability >2.5 avg']=1/data['Avg>2.5']*100

#Implied Probability UNDER 2.5 goals for PINACLE
data['Implied Probability <2.5 pinacle']=1/data['P<2.5']*100

#Implied Probability OVER 2.5 goals for PINACLE
data['Implied Probability >2.5 pinacle']=1/data['P>2.5']*100

#Implied Probability UNDER 2.5 goals for 365
data['Implied Probability <2.5 365']=1/data['B365<2.5']*100

#Implied Probability OVER 2.5 goals for 365
data['Implied Probability >2.5 365']=1/data['B365>2.5']*100

#### Implied Probability Closing

In [10]:
#Implied Probability UNDER 2.5 goals for overall market closing odds (AvgC)
data['Implied Probability <2.5 avg closing']=1/data['AvgC<2.5']*100

#Implied Probability OVER 2.5 goals for overall market closing odds (AvgC)
data['Implied Probability >2.5 avg closing']=1/data['AvgC>2.5']*100

#Implied Probability UNDER 2.5 goals for PINACLE closing odds
data['Implied Probability <2.5 pinacle closing']=1/data['PC<2.5']*100

#Implied Probability OVER 2.5 goals for PINACLE closing odds
data['Implied Probability >2.5 pinacle closing']=1/data['PC>2.5']*100

#Implied Probability UNDER 2.5 goals for 365 closing odds
data['Implied Probability <2.5 365 closing']=1/data['B365C<2.5']*100

#Implied Probability OVER 2.5 goals for 365 closing odds
data['Implied Probability >2.5 365 closing']=1/data['B365C>2.5']*100

### Binning the implied probabilities

#### Binning IP Opening

In [11]:
bins = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100]

#Binning UNDER 2.5 Average Market opening odds
data['binned <2.5 avg'] = pd.cut(data['Implied Probability <2.5 avg'], bins)

#Binning Over 2.5 Average Market opening odds
data['binned >2.5 avg'] = pd.cut(data['Implied Probability >2.5 avg'], bins)

#Binned UNDER 2.5 Pinnacle opening odds
data['binned <2.5 pinacle'] = pd.cut(data['Implied Probability <2.5 pinacle'], bins)

#Binned OVER 2.5 Pinnacle
data['binned >2.5 pinacle'] = pd.cut(data['Implied Probability >2.5 pinacle'], bins)

#Binned UNDER 2.5 bet365 OPENING odds
data['binned <2.5 365'] = pd.cut(data['Implied Probability <2.5 365'], bins)

#Binned OVER 2.5 bet365 OPENING odds
data['binned >2.5 365'] = pd.cut(data['Implied Probability >2.5 365'], bins)

#### Binning IP Closing

In [12]:
bins = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100]

#Binning UNDER 2.5 Average Market closing odds
data['binned <2.5 avg closing'] = pd.cut(data['Implied Probability <2.5 avg closing'], bins)

#Binning OVER 2.5 Average Market closing odds
data['binned >2.5 avg closing'] = pd.cut(data['Implied Probability >2.5 avg closing'], bins)

#Binned UNDER 2.5 Pinnacle closing odds
data['binned <2.5 pinacle closing'] = pd.cut(data['Implied Probability <2.5 pinacle closing'], bins)

#Binned OVER 2.5 Pinnacle CLOSING odds
data['binned >2.5 pinacle closing'] = pd.cut(data['Implied Probability >2.5 pinacle closing'], bins)

#Binned UNDER 2.5 bet365 CLOSING odds
data['binned <2.5 365 closing'] = pd.cut(data['Implied Probability <2.5 365 closing'], bins)

#Binned OVER 2.5 bet365 CLOSING odds
data['binned >2.5 365 closing'] = pd.cut(data['Implied Probability >2.5 365 closing'], bins)

### Binning the odds

#### Binning Odds Opening

In [13]:
bins = [1, 1.5, 2, 3, 99999]

#Binning UNDER 2.5 Average Market opening odds
data['binned odds <2.5 avg'] = pd.cut(data['Avg<2.5'], bins)

#Binning Over 2.5 Average Market opening odds
data['binned odds >2.5 avg'] = pd.cut(data['Avg>2.5'], bins)

#Binned UNDER 2.5 Pinnacle opening odds
data['binned odds <2.5 pinacle'] = pd.cut(data['P<2.5'], bins)

#Binned OVER 2.5 Pinnacle
data['binned odds >2.5 pinacle'] = pd.cut(data['P>2.5'], bins)

#Binned UNDER 2.5 bet365 OPENING odds
data['binned odds <2.5 365'] = pd.cut(data['B365<2.5'], bins)

#Binned OVER 2.5 bet365 OPENING odds
data['binned odds >2.5 365'] = pd.cut(data['B365>2.5'], bins)

#### Binning Odds Closing

In [14]:
bins = [1, 1.5, 2, 3, 99999]

#Binning UNDER 2.5 Average Market opening odds
data['binned odds <2.5 avg closing'] = pd.cut(data['AvgC<2.5'], bins)

#Binning Over 2.5 Average Market opening odds
data['binned odds >2.5 avg closing'] = pd.cut(data['AvgC>2.5'], bins)

#Binned UNDER 2.5 Pinnacle opening odds
data['binned odds <2.5 pinacle closing'] = pd.cut(data['PC<2.5'], bins)

#Binned OVER 2.5 Pinnacle
data['binned odds >2.5 pinacle closing'] = pd.cut(data['PC>2.5'], bins)

#Binned UNDER 2.5 bet365 OPENING odds
data['binned odds <2.5 365 closing'] = pd.cut(data['B365C<2.5'], bins)

#Binned OVER 2.5 bet365 OPENING odds
data['binned odds >2.5 365 closing'] = pd.cut(data['B365C>2.5'], bins)

In [15]:
data['binned odds <2.5 pinacle closing'].isnull().sum()

16

### Other Features from D3

In [16]:
data['Pin_pays_better_under_boolean'] = data['PC<2.5'] > data['AvgC<2.5']

In [17]:
data['Pin_pays_better_under_difference'] = data['PC<2.5'] / data['AvgC<2.5']

In [18]:
data['%vig_p'] = (1 - (1 / (1/data['PC>2.5'] + 1/data['PC<2.5'])))*100

In [19]:
data['%vig_p_bool'] = data['%vig_p']>3.3

In [20]:
data['%vig_avg'] = (1 - (1 / (1/data['AvgC>2.5'] + 1/data['AvgC<2.5'])))*100

In [21]:
data['PC<2.5_P_boolean'] = data['PC<2.5'] < data['P<2.5']

In [22]:
data['PC<2.5_P_relative_diff'] = data['PC<2.5'] / data['P<2.5']

In [23]:
data['MaxC>2.5_AvgC_relative_diff'] = data['MaxC>2.5']/data['AvgC>2.5']
data['Market_consensus'] = data['MaxC>2.5_AvgC_relative_diff']<1.05

In [24]:
## Adding the Year Feature
data_date = data['Date']
data_time = data['Time']

data_date_2 = pd.to_datetime(data_date, dayfirst = True)
data_time_2 = pd.to_datetime(data_time, dayfirst = True)

data['month'] = pd.DatetimeIndex(data_date_2).month
data['month_after_July'] = data['month']>=7

data['year'] = pd.DatetimeIndex(data_date_2).year
data['year_2021_2022'] = data['year']>=2021

data['hour'] = pd.DatetimeIndex(data_time_2).hour
data['game_starts_after_4pm']=data['hour']>=16

In [25]:
data['game_starts_after_4pm'].sum()

6225

### Other Features from Joao

In [26]:
    lst3 = []
    lst4  = []
    for i, team in enumerate(data['AwayTeam']):
        date = data['Date'].iloc[i]
        total2 = len(data[(data['AwayTeam'] == team) & (data['Date'] < date)])
        n_under_away2 = data[(data['AwayTeam'] == team) & (data['Date'] < date)]['under_2.5_goals'].value_counts()
        try:
            lst3.append(1/(n_under_away2[1] / total2))
            lst4.append(n_under_away2[1] / total2)
        except:
            lst3.append(np.nan)
            lst4.append(np.nan)
    data['odds_away_under'] = lst3
    data['prob_away_under'] = lst4

NameError: name 'np' is not defined

# [KILL] Final Dataset [KILL]

### Opening Markets

In [ ]:
#Average Market UNDER opening odds
fdf_under_avg = data[['Implied Probability <2.5 avg','under_2.5_goals','binned <2.5 avg','payout_avg_under_2.5']]

#Average Market OVER opening odds
fdf_over_avg = data[['Implied Probability >2.5 avg','over_2.5_goals','binned >2.5 avg','payout_avg_over_2.5']]

#pinnacle UNDER opening odds
fdf_under_pinacle = data[['Implied Probability <2.5 pinacle','under_2.5_goals','binned <2.5 pinacle','payout_under_2.5_pinacle']]

#pinnacle OVER opening odds
fdf_over_pinacle = data[['Implied Probability >2.5 pinacle','over_2.5_goals','binned >2.5 pinacle','payout_over_2.5_pinacle']]

#bet365 UNDER opening odds
fdf_under_365 = data[['Implied Probability <2.5 365','under_2.5_goals','binned <2.5 365','payout_under_2.5_365']]

#bet365 OVER opening odds
fdf_over_365 = data[['Implied Probability >2.5 365','over_2.5_goals','binned >2.5 365','payout_over_2.5_365']]

### Closing Markets

In [ ]:
#Average Market UNDER closing odds
fdf_under_avg_closing = data[['Implied Probability <2.5 avg closing','under_2.5_goals','binned <2.5 avg closing','payout_avg_under_closing_2.5']]

#Average Market OVER opening odds
fdf_over_avg_closing = data[['Implied Probability >2.5 avg closing','over_2.5_goals','binned >2.5 avg closing','payout_avg_over_closing_2.5']]

#pinnacle UNDER closing odds
fdf_under_pinacle_closing = data[['Implied Probability <2.5 pinacle closing','under_2.5_goals','binned <2.5 pinacle closing','payout_under_2.5_pinacle_closing']]

#pinnacle OVER closing odds
fdf_over_pinacle_closing = data[['Implied Probability >2.5 pinacle closing','over_2.5_goals','binned >2.5 pinacle closing','payout_over_2.5_pinacle_closing']]

#bet365 UNDER closing odds
fdf_under_365_closing = data[['Implied Probability <2.5 365 closing','under_2.5_goals','binned <2.5 365 closing','payout_under_2.5_365_closing']]

#bet365 OVER closing odds
fdf_over_365_closing = data[['Implied Probability >2.5 365 closing','over_2.5_goals','binned >2.5 365 closing','payout_over_2.5_365_closing']]

# [KILL] Results per implied prob Average Market [KILL]

### Under

#### Opening Odds

In [ ]:
fdf_under_avg_agg = fdf_under_avg.groupby('binned <2.5 avg')['payout_avg_under_2.5'].agg(['count', 'mean', 'median'])

fdf_under_pinacle_agg = fdf_under_pinacle.groupby('binned <2.5 pinacle')['payout_under_2.5_pinacle'].agg(['count', 'mean', 'median'])

fdf_under_bet365_agg = fdf_under_365.groupby('binned <2.5 365')['payout_under_2.5_365'].agg(['count', 'mean', 'median'])

In [ ]:
fdf_under_avg_agg.rename(columns={"count": "count_avg", "mean": "mean_avg", "median": "median_avg"}, inplace=True)
fdf_under_avg_agg.index.names = ['bin<2.5']

fdf_under_pinacle_agg.rename(columns={"count": "count_p", "mean": "mean_p", "median": "median_p"}, inplace=True)
fdf_under_pinacle_agg.index.names = ['bin<2.5']

fdf_under_bet365_agg.rename(columns={"count": "count_b365", "mean": "mean_b365", "median": "median_b365"}, inplace=True)
fdf_under_bet365_agg.index.names = ['bin<2.5']

opening_test_1 = fdf_under_avg_agg.merge(fdf_under_pinacle_agg, on='bin<2.5')
opening_test_2 = opening_test_1.merge(fdf_under_bet365_agg, on='bin<2.5')
opening_test_2

#### Closing Odds

In [ ]:
fdf_under_avg_closing_gb_agg = fdf_under_avg_closing.groupby('binned <2.5 avg closing')['payout_avg_under_closing_2.5'].agg(['count', 'mean', 'median', mode])

fdf_under_pinacle_closing_gb_agg = fdf_under_pinacle_closing.groupby('binned <2.5 pinacle closing')['payout_under_2.5_pinacle_closing'].agg(['count', 'mean', 'median', mode])

fdf_under_bet365_closing_gb_agg = fdf_under_365_closing.groupby('binned <2.5 365 closing')['payout_under_2.5_365_closing'].agg(['count', 'mean', 'median', mode])

In [ ]:
fdf_under_avg_closing_gb_agg.rename(columns={"count": "count_avg", "mean": "mean_avg", "median": "median_avg", "mode": "mode_avg"}, inplace=True)
fdf_under_avg_closing_gb_agg.index.names = ['bin<2.5']

fdf_under_pinacle_closing_gb_agg.rename(columns={"count": "count_p", "mean": "mean_p", "median": "median_p", "mode": "mode_p"}, inplace=True)
fdf_under_pinacle_closing_gb_agg.index.names = ['bin<2.5']

fdf_under_bet365_closing_gb_agg.rename(columns={"count": "count_b365", "mean": "mean_b365", "median": "median_b365", "mode": "mode_b365"}, inplace=True)
fdf_under_bet365_closing_gb_agg.index.names = ['bin<2.5']

closing_test_1 = fdf_under_avg_closing_gb_agg.merge(fdf_under_pinacle_closing_gb_agg, on='bin<2.5')
closing_test_2 = closing_test_1.merge(fdf_under_bet365_closing_gb_agg, on='bin<2.5')
closing_test_2['bins'] = closing_test_2.index
closing_test_2

### Over

#### Opening Odds

In [ ]:
fdf_over_avg_agg = fdf_over_avg.groupby('binned >2.5 avg')['payout_avg_over_2.5'].agg(['count', 'mean', 'median'])

fdf_over_pinacle_agg = fdf_over_pinacle.groupby('binned >2.5 pinacle')['payout_over_2.5_pinacle'].agg(['count', 'mean', 'median'])

fdf_over_bet365_agg = fdf_over_365.groupby('binned >2.5 365')['payout_over_2.5_365'].agg(['count', 'mean', 'median'])

In [ ]:
fdf_over_avg_agg.rename(columns={"count": "count_avg", "mean": "mean_avg", "median": "median_avg"}, inplace=True)
fdf_over_avg_agg.index.names = ['bin>2.5']

fdf_over_pinacle_agg.rename(columns={"count": "count_p", "mean": "mean_p", "median": "median_p"}, inplace=True)
fdf_over_pinacle_agg.index.names = ['bin>2.5']

fdf_over_bet365_agg.rename(columns={"count": "count_b365", "mean": "mean_b365", "median": "median_b365"}, inplace=True)
fdf_over_bet365_agg.index.names = ['bin>2.5']

over_opening_test_1 = fdf_over_avg_agg.merge(fdf_over_pinacle_agg, on='bin>2.5')
over_opening_test_2 = over_opening_test_1.merge(fdf_over_bet365_agg, on='bin>2.5')
over_opening_test_2

#### Closing Odds

In [ ]:
fdf_over_avg_closing_agg = fdf_over_avg_closing.groupby('binned >2.5 avg closing')['payout_avg_over_closing_2.5'].agg(['count', 'mean', 'median'])

fdf_over_pinacle_closing_agg = fdf_over_pinacle_closing.groupby('binned >2.5 pinacle closing')['payout_over_2.5_pinacle_closing'].agg(['count', 'mean', 'median'])

fdf_over_bet365_closing_agg = fdf_over_365_closing.groupby('binned >2.5 365 closing')['payout_over_2.5_365_closing'].agg(['count', 'mean', 'median'])

In [ ]:
fdf_over_avg_closing_agg.rename(columns={"count": "count_avg", "mean": "mean_avg", "median": "median_avg"}, inplace=True)
fdf_over_avg_closing_agg.index.names = ['bin>2.5']

fdf_over_pinacle_closing_agg.rename(columns={"count": "count_p", "mean": "mean_p", "median": "median_p"}, inplace=True)
fdf_over_pinacle_closing_agg.index.names = ['bin>2.5']

fdf_over_bet365_closing_agg.rename(columns={"count": "count_b365", "mean": "mean_b365", "median": "median_b365"}, inplace=True)
fdf_over_bet365_closing_agg.index.names = ['bin>2.5']

over_closing_test_1 = fdf_over_avg_closing_agg.merge(fdf_over_pinacle_closing_agg, on='bin>2.5')
over_closing_test_2 = over_closing_test_1.merge(fdf_over_bet365_closing_agg, on='bin>2.5')
over_closing_test_2

# Starting Model

In [27]:
#Pedro's pet! Don't touch it!
"""odd = 2.15
iproba = 1/odd*100
for x in range(0, 20):
    if iproba in closing_test_2.iloc[6]['bins']"""

"odd = 2.15\niproba = 1/odd*100\nfor x in range(0, 20):\n    if iproba in closing_test_2.iloc[6]['bins']"

In [28]:
data_linear_booleans = data.copy()

In [29]:
for col_name in data_linear_booleans.columns: 
    print(col_name)

Div
Date
Time
HomeTeam
AwayTeam
FTHG
FTAG
FTR
HTHG
HTAG
HTR
HS
AS
HST
AST
HF
AF
HC
AC
HY
AY
HR
AR
B365H
B365D
B365A
BWH
BWD
BWA
IWH
IWD
IWA
PSH
PSD
PSA
WHH
WHD
WHA
VCH
VCD
VCA
MaxH
MaxD
MaxA
AvgH
AvgD
AvgA
B365>2.5
B365<2.5
P>2.5
P<2.5
Max>2.5
Max<2.5
Avg>2.5
Avg<2.5
AHh
B365AHH
B365AHA
PAHH
PAHA
MaxAHH
MaxAHA
AvgAHH
AvgAHA
B365CH
B365CD
B365CA
BWCH
BWCD
BWCA
IWCH
IWCD
IWCA
PSCH
PSCD
PSCA
WHCH
WHCD
WHCA
VCCH
VCCD
VCCA
MaxCH
MaxCD
MaxCA
AvgCH
AvgCD
AvgCA
B365C>2.5
B365C<2.5
PC>2.5
PC<2.5
MaxC>2.5
MaxC<2.5
AvgC>2.5
AvgC<2.5
AHCh
B365CAHH
B365CAHA
PCAHH
PCAHA
MaxCAHH
MaxCAHA
AvgCAHH
AvgCAHA
country
Referee
Unnamed: 105
nb_goals
over_2.5_goals
under_2.5_goals
payout_avg_under_2.5
payout_avg_over_2.5
payout_under_2.5_pinacle
payout_over_2.5_pinacle
payout_under_2.5_365
payout_over_2.5_365
payout_avg_under_closing_2.5
payout_avg_over_closing_2.5
payout_under_2.5_pinacle_closing
payout_over_2.5_pinacle_closing
payout_under_2.5_365_closing
payout_over_2.5_365_closing
Implied Probability <2.5 a

In [30]:
data_linear_booleans_lean_P_under = data_linear_booleans[['country','month_after_July','year_2021_2022','game_starts_after_4pm','binned odds <2.5 pinacle closing','Pin_pays_better_under_boolean','Market_consensus','%vig_p_bool','PC<2.5_P_boolean', 'payout_under_2.5_pinacle_closing']]

In [31]:
# data_model_over_under = data_model_over_under[["Div",
# "Date", "Time", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "nb_goals", "Avg>2.5", "Avg<2.5", "B365>2.5", "B365<2.5", "P>2.5", 
# "P<2.5", "AvgC>2.5", "AvgC<2.5", "B365C>2.5", "B365C<2.5", "PC>2.5", "PC<2.5", "over_2.5_goals", "under_2.5_goals", 
# "payout_avg_under_2.5", "payout_avg_over_2.5", "payout_under_2.5_pinacle", "payout_over_2.5_pinacle", "payout_under_2.5_365", 
# "payout_over_2.5_365", "payout_avg_under_closing_2.5", "payout_avg_over_closing_2.5", "payout_under_2.5_pinacle_closing", 
# "payout_over_2.5_pinacle_closing", "payout_under_2.5_365_closing", "payout_over_2.5_365_closing", "Implied Probability <2.5 avg", 
# "Implied Probability >2.5 avg", "Implied Probability <2.5 pinacle", "Implied Probability >2.5 pinacle", "Implied Probability <2.5 365", 
# "Implied Probability >2.5 365", "Implied Probability <2.5 avg closing", "Implied Probability >2.5 avg closing", "Implied Probability <2.5 pinacle closing", 
# "Implied Probability >2.5 pinacle closing", "Implied Probability <2.5 365 closing", "Implied Probability >2.5 365 closing", 
# "binned <2.5 avg", "binned >2.5 avg", "binned <2.5 pinacle", "binned >2.5 pinacle", "binned <2.5 365", "binned >2.5 365", 
# "binned <2.5 avg closing", "binned >2.5 avg closing", "binned <2.5 pinacle closing", "binned >2.5 pinacle closing", 
# "binned <2.5 365 closing", "binned >2.5 365 closing"]]

In [32]:
# data_model_over_under.dropna(inplace=True)

In [33]:
# data_model_over_under

In [34]:
# OneHotEncoders for Bin Odds

In [35]:
ohe = OneHotEncoder(sparse=False) 
ohe.fit(data_linear_booleans_lean_P_under[['binned odds <2.5 pinacle closing']])
bins_encoded = ohe.transform(data_linear_booleans_lean_P_under[['binned odds <2.5 pinacle closing']])
data_linear_booleans_lean_P_under["1.0_to_1.5"], data_linear_booleans_lean_P_under["1.5_to_2.0"], data_linear_booleans_lean_P_under["2.0_to_3.0"], data_linear_booleans_lean_P_under["3.0_to_99999.0"], data_linear_booleans_lean_P_under["extra"] = bins_encoded.T
#data_linear_booleans_lean_P_under.drop(columns='binned <2.5 pinacle', inplace=True)

/tmp/ipykernel_1649/423872295.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_linear_booleans_lean_P_under["1.0_to_1.5"], data_linear_booleans_lean_P_under["1.5_to_2.0"], data_linear_booleans_lean_P_under["2.0_to_3.0"], data_linear_booleans_lean_P_under["3.0_to_99999.0"], data_linear_booleans_lean_P_under["extra"] = bins_encoded.T
/tmp/ipykernel_1649/423872295.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_linear_booleans_lean_P_under["1.0_to_1.5"], data_linear_booleans_lean_P_under["1.5_

In [36]:
ohe = OneHotEncoder(sparse=False) 
ohe.fit(data_linear_booleans_lean_P_under[['country']])
bins_encoded = ohe.transform(data_linear_booleans_lean_P_under[['country']])
data_linear_booleans_lean_P_under["country_1"], data_linear_booleans_lean_P_under["country_2"], data_linear_booleans_lean_P_under["country_3"], data_linear_booleans_lean_P_under["country_4"], data_linear_booleans_lean_P_under["country_5"],data_linear_booleans_lean_P_under["country_6"], data_linear_booleans_lean_P_under["country_7"], data_linear_booleans_lean_P_under["country_8"], data_linear_booleans_lean_P_under["country_9"], data_linear_booleans_lean_P_under["country_10"], data_linear_booleans_lean_P_under["country_11"] = bins_encoded.T
#data_linear_booleans_lean_P_under.drop(columns='binned <2.5 pinacle', inplace=True)

/tmp/ipykernel_1649/2414559662.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_linear_booleans_lean_P_under["country_1"], data_linear_booleans_lean_P_under["country_2"], data_linear_booleans_lean_P_under["country_3"], data_linear_booleans_lean_P_under["country_4"], data_linear_booleans_lean_P_under["country_5"],data_linear_booleans_lean_P_under["country_6"], data_linear_booleans_lean_P_under["country_7"], data_linear_booleans_lean_P_under["country_8"], data_linear_booleans_lean_P_under["country_9"], data_linear_booleans_lean_P_under["country_10"], data_linear_booleans_lean_P_under["country_11"] = bins_encoded.T
/tmp/ipykernel_1649/2414559662.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.

In [37]:
#a =data_linear_booleans_lean_P_under[['country','country_1','country_2','country_3','country_4','country_5','country_6','country_7','country_8','country_9','country_10','country_11']]

In [38]:
#a[a['country_11']==1]

In [39]:
data_linear_booleans_lean_P_under = data_linear_booleans_lean_P_under[data_linear_booleans_lean_P_under['extra']==0]

In [40]:
#extra
data_linear_booleans_lean_P_under = data_linear_booleans_lean_P_under.drop(['country'],axis=1)

In [41]:
#extra
data_linear_booleans_lean_P_under = data_linear_booleans_lean_P_under.drop(['binned odds <2.5 pinacle closing'],axis=1)

In [42]:
data_linear_booleans_lean_P_under.sum()

month_after_July                     5346.00
year_2021_2022                       5732.00
game_starts_after_4pm                6221.00
Pin_pays_better_under_boolean        9222.00
Market_consensus                     6072.00
%vig_p_bool                          3755.00
PC<2.5_P_boolean                     4932.00
payout_under_2.5_pinacle_closing    10100.48
1.0_to_1.5                            440.00
1.5_to_2.0                           4587.00
2.0_to_3.0                           4781.00
3.0_to_99999.0                        620.00
extra                                   0.00
country_1                             844.00
country_2                             909.00
country_3                            1140.00
country_4                            1039.00
country_5                             720.00
country_6                            1140.00
country_7                             841.00
country_8                             917.00
country_9                             634.00
country_10

In [43]:
for col_name in data_linear_booleans_lean_P_under.columns: 
    print(col_name)

month_after_July
year_2021_2022
game_starts_after_4pm
Pin_pays_better_under_boolean
Market_consensus
%vig_p_bool
PC<2.5_P_boolean
payout_under_2.5_pinacle_closing
1.0_to_1.5
1.5_to_2.0
2.0_to_3.0
3.0_to_99999.0
extra
country_1
country_2
country_3
country_4
country_5
country_6
country_7
country_8
country_9
country_10
country_11


In [44]:
# test_model = data_model_over_under.drop(columns=['FTHG', 'FTAG', 'nb_goals', 'over_2.5_goals', 'under_2.5_goals', 
#                                                 'Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam', 'binned <2.5 avg', 
#                                                 'binned >2.5 avg', 'binned >2.5 pinacle', 'binned <2.5 365', 
#                                                  'binned >2.5 365', 'binned <2.5 avg closing', 'binned >2.5 avg closing', 
#                                                 'binned <2.5 pinacle closing', 'binned >2.5 pinacle closing', 'binned <2.5 365 closing', 
#                                                 'binned >2.5 365 closing', 
#                                                 'B365>2.5', 'B365<2.5', 'B365C>2.5', 'B365C<2.5', 'payout_under_2.5_365', 'payout_over_2.5_365', 
#                                                 'payout_under_2.5_365_closing', 'payout_over_2.5_365_closing', 
#                                                 'Implied Probability <2.5 365', 'Implied Probability >2.5 365', 
#                                                 'Implied Probability <2.5 365 closing', 'Implied Probability >2.5 365 closing', 
#                                                 'payout_avg_under_2.5', 'payout_avg_over_2.5', 'payout_under_2.5_pinacle', 
#                                                 'payout_avg_over_closing_2.5', 'payout_over_2.5_pinacle_closing', 
#                                                 'Implied Probability <2.5 avg', 'Implied Probability >2.5 avg', 
#                                                 'Implied Probability <2.5 pinacle', 'Implied Probability >2.5 pinacle', 
#                                                 'Avg>2.5', 'P>2.5', 'AvgC>2.5', 'PC>2.5', 'payout_over_2.5_pinacle', 
#                                                 'Implied Probability >2.5 avg closing', 'Implied Probability >2.5 pinacle closing', 
#                                                 'payout_avg_under_closing_2.5', 'Avg<2.5', 'P<2.5', 'AvgC<2.5', 'PC<2.5', 
#                                                 'Implied Probability <2.5 avg closing', 'Implied Probability <2.5 pinacle closing'])

# test_model

In [45]:
data_linear_booleans_lean_P_under.columns

Index(['month_after_July', 'year_2021_2022', 'game_starts_after_4pm',
       'Pin_pays_better_under_boolean', 'Market_consensus', '%vig_p_bool',
       'PC<2.5_P_boolean', 'payout_under_2.5_pinacle_closing', '1.0_to_1.5',
       '1.5_to_2.0', '2.0_to_3.0', '3.0_to_99999.0', 'extra', 'country_1',
       'country_2', 'country_3', 'country_4', 'country_5', 'country_6',
       'country_7', 'country_8', 'country_9', 'country_10', 'country_11'],
      dtype='object')

In [46]:
X = data_linear_booleans_lean_P_under[['Pin_pays_better_under_boolean','Market_consensus','%vig_p_bool','PC<2.5_P_boolean', '1.0_to_1.5', '1.5_to_2.0', '2.0_to_3.0','3.0_to_99999.0']]
X_step2 = data_linear_booleans_lean_P_under[['%vig_p_bool','PC<2.5_P_boolean', '1.0_to_1.5', '1.5_to_2.0', '2.0_to_3.0','3.0_to_99999.0']]
X_step3 = data_linear_booleans_lean_P_under[['country_1','country_2','country_3','country_4','country_5','country_6','country_7','country_8','country_9','country_10','country_11','month_after_July', 'year_2021_2022', 'game_starts_after_4pm','Pin_pays_better_under_boolean','Market_consensus','%vig_p_bool','PC<2.5_P_boolean','1.0_to_1.5', '1.5_to_2.0', '2.0_to_3.0','3.0_to_99999.0']]
#X_step4 = data_linear_booleans_lean_P_under[['country_1','country_2','country_3','country_4','country_5','country_6','country_7','country_8','country_9','country_10','country_11']]
y = data_linear_booleans_lean_P_under['payout_under_2.5_pinacle_closing']

In [47]:
#X_step4 = data_linear_booleans_lean_P_under[['country_1','country_2','country_3','country_4','country_5','country_6','country_7','country_8','country_9','country_10','country_11']]
#y = data_linear_booleans_lean_P_under['payout_under_2.5_pinacle_closing']

In [48]:
import statsmodels.api as sm

In [49]:
results = sm.OLS(y,X_step3.astype(float)).fit()

In [50]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                   OLS Regression Results                                   
============================================================================================
Dep. Variable:     payout_under_2.5_pinacle_closing   R-squared:                       0.002
Model:                                          OLS   Adj. R-squared:                 -0.000
Method:                               Least Squares   F-statistic:                    0.8333
Date:                              Thu, 02 Jun 2022   Prob (F-statistic):              0.675
Time:                                      22:31:43   Log-Likelihood:                -15332.
No. Observations:                             10428   AIC:                         3.071e+04
Df Residuals:                                 10407   BIC:                         3.086e+04
Df Model:                                        20                                         
Covariance Type:                          nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
country_1                         0.2311      0.041      5.574      0.000       0.150       0.312
country_2                         0.2083      0.037      5.697      0.000       0.137       0.280
country_3                         0.2809      0.033      8.512      0.000       0.216       0.346
country_4                         0.2203      0.034      6.421      0.000       0.153       0.288
country_5                         0.2334      0.047      5.007      0.000       0.142       0.325
country_6                         0.2070      0.034      6.129      0.000       0.141       0.273
country_7                         0.2935      0.039      7.508      0.000       0.217       0.370
country_8                         0.2818      0.038      7.446      0.000       0.208       0.356
country_9                         0.2619      0.043      6.130      0.000       0.178       0.346
country_10                        0.2888      0.033      8.753      0.000       0.224       0.353
country_11                        0.2423      0.035      7.018      0.000       0.175       0.310
month_after_July                 -0.0255      0.023     -1.103      0.270      -0.071       0.020
year_2021_2022                   -0.0073      0.023     -0.313      0.754      -0.053       0.038
game_starts_after_4pm             0.0286      0.022      1.299      0.194      -0.015       0.072
Pin_pays_better_under_boolean    -0.0014      0.034     -0.041      0.967      -0.069       0.066
Market_consensus                  0.0004      0.023      0.017      0.987      -0.045       0.046
%vig_p_bool                       0.0126      0.030      0.419      0.675      -0.046       0.071
PC<2.5_P_boolean                  0.0315      0.022      1.414      0.157      -0.012       0.075
1.0_to_1.5                        0.6879      0.060     11.453      0.000       0.570       0.806
1.5_to_2.0                        0.7007      0.036     19.222      0.000       0.629       0.772
2.0_to_3.0                        0.7073      0.039     18.175      0.000       0.631       0.784
3.0_to_99999.0                    0.6534      0.058     11.302      0.000       0.540       0.767
==============================================================================
Omnibus:                    18945.660   Durbin-Watson:                   1.992
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              961.422
Skew:                           0.393   Prob(JB):                    1.70e-209
Kurtosis:                       1.737   Cond. No.                     5.41e+15
==============================================================================

Note

In [51]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

data = pd.read_csv('../input/fifa-2018-match-statistics/FIFA 2018 Statistics.csv')
y = (data['Man of the Match'] == "Yes")  # Convert from string "Yes"/"No" to binary
feature_names = [i for i in data.columns if data[i].dtype in [np.int64, np.int64]]
X = data[feature_names]
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)
my_model = RandomForestClassifier(random_state=0).fit(train_X, train_y)

FileNotFoundError: [Errno 2] No such file or directory: '../input/fifa-2018-match-statistics/FIFA 2018 Statistics.csv'

In [52]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
my_model = RandomForestRegressor(random_state=0).fit(X_step3, y)

In [ ]:
pip install shap

In [ ]:
y_pred = my_model.predict(X_step3)

In [69]:
import sklearn

In [ ]:
r2train = sklearn.metrics.r2_score(y, y_pred)

In [ ]:
r2train

In [ ]:
#import shap  # package used to calculate Shap values

# Create object that can calculate shap values
explainer = shap.TreeExplainer(my_model)

# calculate shap values. This is what we will plot.
# Calculate shap_values for all of val_X rather than a single row, to have more data for plot.
shap_values = explainer.shap_values(X_step3)

# Make plot. Index of [1] is explained in text below.
shap.summary_plot(shap_values, X_step3)

#- Vertical location shows what feature it is depicting
#- Color shows whether that feature was high or low for that row of the dataset
#- Horizontal location shows whether the effect of that value caused a higher or lower prediction.

In [ ]:
## Each dot represents a row of the data. The horizontal location is the actual value from the dataset, and the vertical location shows what having that value did to the prediction.
shap.dependence_plot('%vig_p_bool', shap_values, X_step3, dot_size=10)

In [ ]:
#shap.summary_plot(shap_values, X_step3, plot_type="bar")

In [ ]:
pip install XGBoost

In [60]:
import xgb as xgb

ModuleNotFoundError: No module named 'xgb'

In [61]:
m = xgb.XGBRegressor()

In [65]:
X_step3.rename(columns = {'PC<2.5_P_boolean':'PC_under_2.5_P_boolean'}, inplace = True)

/tmp/ipykernel_1649/3693275891.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_step3.rename(columns = {'PC<2.5_P_boolean':'PC_under_2.5_P_boolean'}, inplace = True)


In [64]:
X_step3.columns

Index(['country_1', 'country_2', 'country_3', 'country_4', 'country_5',
       'country_6', 'country_7', 'country_8', 'country_9', 'country_10',
       'country_11', 'month_after_July', 'year_2021_2022',
       'game_starts_after_4pm', 'Pin_pays_better_under_boolean',
       'Market_consensus', '%vig_p_bool', 'PC<2.5_P_boolean', '1.0_to_1.5',
       '1.5_to_2.0', '2.0_to_3.0', '3.0_to_99999.0'],
      dtype='object')

In [66]:
m.fit(X_step3,y) 

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [67]:
y_pred = m.predict(X_step3)

In [70]:
r2train = sklearn.metrics.r2_score(y, y_pred)

In [71]:
r2train

0.12647703579056846

In [ ]:
import XGBoost  # package used to calculate Shap values

# Create object that can calculate shap values
explainer = shap.TreeExplainer(my_model)

# calculate shap values. This is what we will plot.
# Calculate shap_values for all of val_X rather than a single row, to have more data for plot.
shap_values = explainer.shap_values(X_step3)

# Make plot. Index of [1] is explained in text below.
shap.summary_plot(shap_values, X_step3)

#- Vertical location shows what feature it is depicting
#- Color shows whether that feature was high or low for that row of the dataset
#- Horizontal location shows whether the effect of that value caused a higher or lower prediction.

In [ ]:
data_linear_booleans_lean_P_under = data_linear_booleans_lean_P_under[data_linear_booleans_lean_P_under['payout_under_2.5_pinacle'].isnull()==False]

In [ ]:
y

In [ ]:
model = LinearRegression()
model_fit = model.fit(X, y)

In [ ]:
result = sm.ols(formula='payout_under_2 ~ Pin_pays_better_under_boolean + PC<2.5_P_boolean + 1.0_to_1.5 + 1.5_to_2.0 + 2.0_to_3.0 + 3.0_to_99999.0', data=data_linear_booleans_lean_P_under).fit()
 
    #smf.ols(formula='weight ~ horsepower + cylinders', data=mpg).fit()
    
# printing the summary table
print(result.summary())

In [ ]:
model_fit.summary()

In [ ]:
#Features that decrease our chances
for x in [2, 1, 0, 13, 4, 12, 11]:
    print(f'{X.columns[x]}: {importance[x]}')

In [ ]:
#Features that increase our chances
for x in [6, 8, 7, 5, 9, 10, 3]:
    print(f'{X.columns[x]}: {importance[x]}')